In [42]:
import os

In [43]:
%pwd

'/Users/rupeshyadav/Documents/MLFLOW/End-To-End-MLOPs-Credit_Card_Fraud_Detection'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/rupeshyadav/Documents/MLFLOW/End-To-End-MLOPs-Credit_Card_Fraud_Detection'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/Rupesh-132/End-To-End-MLOPs-Credit_Card_Fraud_Detection.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "Rupesh-132"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "d0abd907d6e7b5b15f194e16de7c132a9ef9cdef"

Entity Definition for model evaluation

In [13]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


Configuration Manager for model evaluation strategy

In [14]:
from Credit_Card_Fraud_Detection.constants import *
from Credit_Card_Fraud_Detection.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/Rupesh-132/End-To-End-MLOPs-Credit_Card_Fraud_Detection.mlflow"
           
        )

        return model_evaluation_config


In [16]:
import os
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score,precision_score,recall_score,f1_score,matthews_corrcoef
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from Credit_Card_Fraud_Detection.utils.common import save_json

In [17]:
class ModelEvaluationStrategy:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        #classification_report = classification_report(actual,pred)
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred)
        recall = recall_score(actual,pred)
        f1 = f1_score(actual,pred)
        matthews_corr = matthews_corrcoef(actual,pred)
        
        return accuracy,precision,recall,f1,matthews_corr



    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (accuracy,precision,recall,f1,matthews_corr) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"accuracy": accuracy, "precision": precision, "recall_score": recall,"f1_score":f1,"matthews_corr":matthews_corr}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            
            

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall_score", recall)
            mlflow.log_metric("f1_score",f1)
            mlflow.log_metric("matthews_corrcoef",matthews_corr)
            

        
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="RandomForestClassifier")
            else:
                mlflow.sklearn.log_model(model, "model")
    
     


    


In [18]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluationStrategy(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-01-05 12:07:08,450: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-05 12:07:08,456: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-05 12:07:08,460: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-05 12:07:08,462: INFO: common: created directory at: artifacts]
[2024-01-05 12:07:08,463: INFO: common: created directory at: artifacts/model_evaluation]
[2024-01-05 12:08:23,711: WARNING: connectionpool: Retrying (Retry(total=4, connect=4, read=5, redirect=5, status=5)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x286d2d4d0>, 'Connection to dagshub.com timed out. (connect timeout=120)')': /Rupesh-132/End-To-End-MLOPs-Credit_Card_Fraud_Detection.mlflow/api/2.0/mlflow/runs/create]
[2024-01-05 12:08:27,995: WARNING: connectionpool: Retrying (Retry(total=3, connect=3, read=5, redirect=5, status=5)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConne

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.1s
/Users/rupeshyadav/Documents/MLFLOW/venv/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/rupeshyadav/Documents/MLFLOW/venv/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'RandomForestClassifier' already exists. Creating a new version of this model...
2024/01/05 12:08:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to f

In [19]:
import joblib


In [20]:
model = joblib.load(Path("artifacts/model_trainer/model.joblib"))

In [40]:
[170348.0, 1.9919760961759, 0.158475887304227, -2.58344064503516, 0.408669992998441, 1.15114706077937, -0.0966947441848027, 0.223050267455537, -0.0683838777747007, 0.577829383844873, -0.888721675865145, 0.491140241656789, 0.728903319843614, 0.380428045513993, -1.94888334870021, -0.832498136300872, 0.519435549203291, 0.903562376617253, 1.19731471799372, 0.593508846946918, -0.0176522567052908, -0.164350327825504, -0.295135166851559, -0.0721725311018398, -0.450261313423321, 0.313266608995469, -0.289616585696882, 0.002987582243429, -0.0153088128485981, 0.0]

In [41]:
model.predict(np.array(data).reshape(1,30))

/Users/rupeshyadav/Documents/MLFLOW/venv/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


array([0])